## Import Libraries 

In [3]:
import sys
sys.path.append('/paths_generator.py')
import paths_generator as pg
import random as rand
import math as m
import numpy as np

## Simulated Annealing Function

In [1]:
def simulated_annealing (init_st, cost_func, max_it, max_T, cooling_rate):   #cooling_rate howa li ki khli T decreasa
  current_st = init_st
  best_st = init_st
  T = max_T

  for i in range(max_it):
    if T <= 0:   
      break

    new_st = current_st.get_neighbor()   # Get a random neighbor of the current state
    current_C = cost_func(current_st)
    new_C = cost_func(new_C)

    if new_C < current_C:
      current_st = new_st # Accept the new state as the current state
      if new_C < cost_func(best_st):
        best_state = new_st  # Update the best state if the new state has a minimal cost
    
    else: 
      proba = m.exp(- (current_C - new_C) / T)
      if rand.random() < proba:
        current_st = new_st
        T *= cooling_rate # Cool down the temperature for the next iteration

  return best_st

## Utils

Init data

In [ ]:
zones_num = pg.region.shape[0]*pg.region.shape[1]

 Check sign

In [1]:
def check_sign(lst):

    for ele in lst:
        
        if ele > 0:
            return False
        
    return True

## Opti problem

#### Objective function

In [3]:
def objective(x):  #m, n depends on the path: temperary

  S = 0
  
  for i in range(zones_num): # m: present the number of places where we will put our edge
    e_i = pg.region[pg.Rcount(i)]  # elevation of the zone i

    for j in range(zones_num): # n: present the number of places that we will prevet water flow from thanks to the edge
      S += x[i][j] * (pg.slr - e_i)  # maybe direcly ndiro og.
  
  return S

#### Constraint Function

In [ ]:
# It works just for one asset
def constraint(x): 
    
    constraints = []

    for asset in pg.assets:
      for road in pg.T_Ci[asset]:
        temp = []
        
        for i in range(len(road)-1):
          temp.append(x[road[i+1]][road[i]])

        constraints.append(1-sum(temp))   # All of them should be < 0
    
    return check_sign(constraint)